# LangChain Playground

## Init Model and use some prompt template

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import GPT4All

import config

In [3]:
template = "Tell me the DnD rule for the follow: {action}"
prompt = PromptTemplate(template=template, input_variables=['action'])

In [4]:
callbacks = [StreamingStdOutCallbackHandler()]
llm = GPT4All(model=config.gpt_4all_groovy, backend='gptj', callbacks=callbacks)

Found model file.


In [5]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
llm_chain.run('Spellcasting')

''

## Using indexers for IR

### Convert text pages to embeddings and store in vectorstore

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [8]:
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [14]:
from langchain.document_loaders import PyPDFLoader
import os

if not os.path.exists('phb_faiss_index'):
    loader = PyPDFLoader("C:\\Users\\phili\\Documents\\DND_5E_PHB.pdf")
    documents = loader.load()

In [10]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

if not os.path.exists('phb_faiss_index'):
    print('Re-Indexing Player Handbook')

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=32)
    pages = text_splitter.split_documents(documents)

    faiss_index = FAISS.from_documents(pages, embeddings)
    faiss_index.save_local('phb_faiss_index')
else:
    faiss_index = FAISS.load_local('phb_faiss_index', embeddings)

## Combine Language model and Index Store to get a Document Question Answering Chain

In [27]:
template = """
Use the following context to answer questions.
Context: {context}
---
The following question is about the tabletop game Dungeons and Dragons.
Question: {question}
Answer: Let's think step by step.
"""

In [29]:
question_prompt_template = PromptTemplate(template=template, input_variables=['question', 'context'])
llm_qa_chain = LLMChain(llm=llm, prompt=question_prompt_template)

def get_answer_for_question(question: str) -> str:
    similar_documents = faiss_index.similarity_search(question, k=2)
    context = "\n".join([d.page_content for d in similar_documents])
    return llm_qa_chain.run(context=context, question=question)

In [30]:
get_answer_for_question('how do I level up?')

1) Level 1 to 2 - Experience points are earned when you complete an adventure or defeat a monster in combat (depending on your character class). You can earn these experience points through playing mini-games, completing quests and solving puzzles within the game world. Once this first level is reached, choose one option from table A:A) Use all three of them to upgrade some equipment  for 3rd or 4th levels(e.) Upgrade a weapon (D), armor (C), shield +1d6 damage at 6HD; and 1 bonus spell per caster level. You can use these new items in the future, whenever you're ready - just keep track!B) Don't upgrade any equipment until 3rd or 4th levels(e.) If your experience points are enough to gain a certain number of "levels" (experience point milestones), choose one option from table B:  I. You can use these new items when the time comes, whenever you're ready - just keep track!2) Level 2 to 3 - As soon as possible after gaining levels 1 and/or 2 by

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x0000025CAC91DB40>
Traceback (most recent call last):
  File "C:\Users\phili\miniconda3\envs\torch_python_310\lib\site-packages\gpt4all\pyllmodel.py", line 239, in _response_callback
    @staticmethod
KeyboardInterrupt: 



1) Level 1 to 2 - Experience points are earned when you complete an adventure or defeat a monster in combat (depending on your character class). You can earn these experience points through playing mini-games, completing quests and solving puzzles within the game world. Once this first level is reached, choose one option from table A:A) Use all three of them to upgrade some equipment  for 3rd or 4th levels(e.) Upgrade a weapon (D), armor (C), shield +1d6 damage at 6HD; and 1 bonus spell per caster level. You can use these new items in the future, whenever you're ready - just keep track!B) Don't upgrade any equipment until 3rd or 4th levels(e.) If your experience points are enough to gain a certain number of "levels" (experience point milestones), choose one option from table B:  I. You can use these new items when the time comes, whenever you're ready - just keep track!2) Level 2 to 3 - As soon as possible after gaining levels 1 and/or 2 by

'1) Level 1 to 2 - Experience points are earned when you complete an adventure or defeat a monster in combat (depending on your character class). You can earn these experience points through playing mini-games, completing quests and solving puzzles within the game world. Once this first level is reached, choose one option from table A:A) Use all three of them to upgrade some equipment  for 3rd or 4th levels(e.) Upgrade a weapon (D), armor (C), shield +1d6 damage at 6HD; and 1 bonus spell per caster level. You can use these new items in the future, whenever you\'re ready - just keep track!B) Don\'t upgrade any equipment until 3rd or 4th levels(e.) If your experience points are enough to gain a certain number of "levels" (experience point milestones), choose one option from table B:  I. You can use these new items when the time comes, whenever you\'re ready - just keep track!2) Level 2 to 3 - As soon as possible after gaining levels 1 and/or 2 by'